<a href="https://colab.research.google.com/github/Navyasaipusala/quiz1/blob/main/q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
import keras_tuner as kt

# Load dataset (example: CIFAR-10)
(x_train, y_train), (x_val, y_val) = keras.datasets.cifar10.load_data()
x_train, x_val = x_train / 255.0, y_val / 255.0  # Normalize

# Function to define a model with hyperparameters
def build_model(hp):
    model = keras.Sequential()

    # First Convolutional Layer
    model.add(layers.Conv2D(
        filters=hp.Int('conv1_filters', min_value=32, max_value=128, step=32),
        kernel_size=hp.Choice('conv1_kernel', values=[3, 5]),
        activation='relu',
        kernel_regularizer=regularizers.l2(1e-4),
        input_shape=(32, 32, 3)
    ))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=2))

    # Second Convolutional Layer
    model.add(layers.Conv2D(
        filters=hp.Int('conv2_filters', min_value=64, max_value=256, step=64),
        kernel_size=hp.Choice('conv2_kernel', values=[3, 5]),
        activation='relu',
        kernel_regularizer=regularizers.l2(1e-4)
    ))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=2))

    # Flatten Layer
    model.add(layers.Flatten())

    # Fully Connected Layer
    model.add(layers.Dense(
        units=hp.Int('dense_units', min_value=64, max_value=512, step=64),
        activation='relu',
        kernel_regularizer=regularizers.l2(1e-4)
    ))
    model.add(layers.Dropout(hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1)))

    # Output Layer
    model.add(layers.Dense(10, activation='softmax'))

    # Compile Model
    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Use Keras Tuner for Hyperparameter Search
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    directory='tuner_results',
    project_name='cifar10_tuning'
)

# Perform Hyperparameter Search
tuner.search(x_train, y_train, epochs=10, validation_data=(x_val, y_val))

# Get Best Model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best Hyperparameters: {best_hps.values}")

# Train Best Model with Early Stopping
best_model = tuner.hypermodel.build(best_hps)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = best_model.fit(
    x_train, y_train,
    epochs=50,
    validation_data=(x_val, y_val),
    batch_size=32,
    callbacks=[early_stopping]
)

# Evaluate Model
test_loss, test_acc = best_model.evaluate(x_val, y_val)
print(f"Validation Accuracy: {test_acc:.4f}")
print(f"Validation Loss: {test_loss:.4f}")


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
import keras_tuner as kt

# Step 1: Load and Preprocess the Dataset
# - Normalization: Scaling pixel values (0 to 255) to range [0,1] for stable training.
(x_train, y_train), (x_val, y_val) = keras.datasets.cifar10.load_data()
x_train, x_val = x_train / 255.0, x_val / 255.0

# Step 2: Define the Hyperparameter Search Model
def build_model(hp):
    """
    Model Structure with Improvements:
    1. Convolutional Layers:
        - Increased filter count dynamically for feature extraction.
        - Used different kernel sizes for better spatial hierarchy learning.
    2. Regularization:
        - L2 Regularization to prevent overfitting.
        - Batch Normalization to stabilize activations and speed up convergence.
    3. Fully Connected Layers:
        - Tuned dense units for balancing complexity and efficiency.
        - Applied dropout for preventing co-adaptation of neurons.
    4. Learning Rate Optimization:
        - Experimented with different learning rates to optimize convergence.
    """

    model = keras.Sequential()

    # Convolutional Layer 1: Extracts basic patterns (edges, textures)
    model.add(layers.Conv2D(
        filters=hp.Int('conv1_filters', 32, 128, step=32),
        kernel_size=hp.Choice('conv1_kernel', [3, 5]),
        activation='relu',
        kernel_regularizer=regularizers.l2(1e-4),
        input_shape=(32, 32, 3)
    ))
    model.add(layers.BatchNormalization())  # Stabilizes training
    model.add(layers.MaxPooling2D(pool_size=2))  # Reduces spatial size

    # Convolutional Layer 2: Extracts complex patterns
    model.add(layers.Conv2D(
        filters=hp.Int('conv2_filters', 64, 256, step=64),
        kernel_size=hp.Choice('conv2_kernel', [3, 5]),
        activation='relu',
        kernel_regularizer=regularizers.l2(1e-4)
    ))
    model.add(layers.BatchNormalization())  # Normalization for consistency
    model.add(layers.MaxPooling2D(pool_size=2))

    # Fully Connected Layer: Decision making
    model.add(layers.Flatten())
    model.add(layers.Dense(
        units=hp.Int('dense_units', 64, 512, step=64),
        activation='relu',
        kernel_regularizer=regularizers.l2(1e-4)
    ))
    model.add(layers.Dropout(hp.Float('dropout', 0.2, 0.5, step=0.1)))  # Prevents overfitting

    # Output Layer: Classification into 10 categories
    model.add(layers.Dense(10, activation='softmax'))

    # Optimizer: Adaptive optimization with tunable learning rate
    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Step 3: Hyperparameter Optimization
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    directory='tuner_results',
    project_name='cifar10_tuning'
)

# Search for Best Hyperparameters
tuner.search(x_train, y_train, epochs=10, validation_data=(x_val, y_val))

# Step 4: Train the Best Model with Optimized Hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hps)

# Step 5: Apply Early Stopping for Efficient Training
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = best_model.fit(
    x_train, y_train,
    epochs=50,
    validation_data=(x_val, y_val),
    batch_size=32,
    callbacks=[early_stopping]
)

# Step 6: Model Evaluation and Impact Analysis
test_loss, test_acc = best_model.evaluate(x_val, y_val)
print(f"Validation Accuracy: {test_acc:.4f}")
print(f"Validation Loss: {test_loss:.4f}")

# Logical Explanation of Impact:
impact_description = """
1. Hyperparameter tuning enabled *adaptive architecture refinement*, leading to improved accuracy.
2. *Batch Normalization* reduced internal covariate shift, stabilizing training.
3. *L2 Regularization* prevented overfitting by controlling weight magnitudes.
4. *Dropout* reduced over-reliance on specific neurons, improving generalization.
5. *Learning rate tuning* optimized model convergence speed and final accuracy.
6. *Early stopping* ensured efficiency by avoiding unnecessary epochs.
"""
print(impact_description)


In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Assuming 'history' is the training history returned by model.fit()
# Extract accuracy and loss values
epochs = range(1, len(history.history['accuracy']) + 1)

#  Visualization 1: Training & Validation Accuracy
plt.figure(figsize=(8, 6))
plt.plot(epochs, history.history['accuracy'], 'bo-', label='Training Accuracy')
plt.plot(epochs, history.history['val_accuracy'], 'r^-', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training vs Validation Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Visualization 2: Confusion Matrix
# Get model predictions
y_pred = np.argmax(best_model.predict(x_val), axis=1)
y_true = y_val.flatten()

# Compute confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Plot the confusion matrix using Seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=range(10), yticklabels=range(10))
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

#  Step 1: Load the Dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

#  Step 2: Preprocess the Data
# Normalize pixel values to range [0, 1]
x_train, x_test = x_train / 255.0, x_test / 255.0

# Reshape images to include channel dimension (28x28x1)
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

#  Step 3: Define the Baseline CNN Model
def build_baseline_model():
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D(2, 2),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')  # 10 classes
    ])

    # Compile model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Step 4: Train the Model
baseline_model = build_baseline_model()

history = baseline_model.fit(
    x_train, y_train,
    epochs=10,
    validation_data=(x_test, y_test),
    batch_size=32
)

#  Step 5: Evaluate Model Performance
test_loss, test_acc = baseline_model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test Loss: {test_loss:.4f}")

#  Step 6: Visualize Training & Validation Accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training vs Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np

#  Step 1: Load and Preprocess Dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

# Normalize pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

#  Step 2: Define the Modified CNN Model
def build_modified_model():
    model = keras.Sequential([
        # First Conv Block
        layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(1e-4), input_shape=(28, 28, 1)),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),

        # Second Conv Block
        layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),

        # Third Conv Block
        layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),

        # Flatten & Fully Connected Layers
        layers.Flatten(),
        layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
        layers.Dropout(0.3),  # Prevents overfitting
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(10, activation='softmax')  # Output Layer for 10 classes
    ])

    # Compile model
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

#  Step 3: Train the Modified Model
modified_model = build_modified_model()

history = modified_model.fit(
    x_train, y_train,
    epochs=20,
    validation_data=(x_test, y_test),
    batch_size=32
)

#  Step 4: Evaluate Model Performance
test_loss, test_acc = modified_model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test Loss: {test_loss:.4f}")

#  Step 5: Visualize Training & Validation Accuracy
plt.figure(figsize=(8, 6))
plt.plot(history.history['accuracy'], 'bo-', label='Training Accuracy')
plt.plot(history.history['val_accuracy'], 'r^-', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training vs Validation Accuracy')
plt.legend()
plt.grid(True)
plt.show()

#  Step 6: Confusion Matrix Visualization
y_pred = np.argmax(modified_model.predict(x_test), axis=1)
y_true = y_test.flatten()
conf_matrix = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=range(10), yticklabels=range(10))
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

#  Step 1: Evaluate on Test Set
test_loss, test_acc = modified_model.evaluate(x_test, y_test)
print(f"\n Test Accuracy: {test_acc:.4f}")
print(f" Test Loss: {test_loss:.4f}")

# Step 2: Generate Predictions
y_pred = np.argmax(modified_model.predict(x_test), axis=1)
y_true = y_test.flatten()

# Step 3: Classification Report
print("\n Classification Report:")
print(classification_report(y_true, y_pred, target_names=[
    "T-shirt/Top", "Trouser", "Pullover", "Dress", "Coat",
    "Sandal", "Shirt", "Sneaker", "Bag", "Ankle Boot"
]))

# Step 4: Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=range(10), yticklabels=range(10))
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

#  Step 1: Save the Improved Model
model_path = "fashion_mnist_improved_model.h5"
modified_model.save(model_path)
print(f"Model saved at: {model_path}")

# Step 2: Load the Saved Model
loaded_model = tf.keras.models.load_model(model_path)
print(" Model loaded successfully!")

#  Step 3: Use the Model for Predictions
# Select random images from the test set
num_samples = 5
indices = np.random.choice(len(x_test), num_samples, replace=False)
sample_images = x_test[indices]
sample_labels = y_test[indices]

# Predict class probabilities
predictions = loaded_model.predict(sample_images)
predicted_labels = np.argmax(predictions, axis=1)

#  Step 4: Display Predictions
class_names = [
    "T-shirt/Top", "Trouser", "Pullover", "Dress", "Coat",
    "Sandal", "Shirt", "Sneaker", "Bag", "Ankle Boot"
]

plt.figure(figsize=(10, 5))
for i in range(num_samples):
    plt.subplot(1, num_samples, i+1)
    plt.imshow(sample_images[i].reshape(28, 28), cmap='gray')
    plt.title(f"Pred: {class_names[predicted_labels[i]]}\nTrue: {class_names[sample_labels[i]]}")
    plt.axis("off")

plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

#  Step 1: Generate Predictions
y_pred = np.argmax(loaded_model.predict(x_test), axis=1)
y_true = y_test.flatten()

#  Step 2: Compute Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)

#  Step 3: Plot Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=range(10), yticklabels=range(10))
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

#  Step 1: Extract Training History
history_dict = history.history

#  Step 2: Create Subplots
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

# Plot Accuracy
ax[0].plot(history_dict['accuracy'], 'bo-', label='Training Accuracy')
ax[0].plot(history_dict['val_accuracy'], 'r^-', label='Validation Accuracy')
ax[0].set_title('Training vs Validation Accuracy')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Accuracy')
ax[0].legend()
ax[0].grid(True)

# Plot Loss
ax[1].plot(history_dict['loss'], 'bo-', label='Training Loss')
ax[1].plot(history_dict['val_loss'], 'r^-', label='Validation Loss')
ax[1].set_title('Training vs Validation Loss')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Loss')
ax[1].legend()
ax[1].grid(True)

#  Step 3: Display the Plot
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#  Step 1: Compute Class-wise Accuracy
y_pred = np.argmax(loaded_model.predict(x_test), axis=1)
y_true = y_test.flatten()
class_accuracy = [(y_pred[y_true == i] == i).mean() for i in range(10)]

#  Step 2: Define Class Names
class_names = [
    "T-shirt/Top", "Trouser", "Pullover", "Dress", "Coat",
    "Sandal", "Shirt", "Sneaker", "Bag", "Ankle Boot"
]

#  Step 3: Plot Class-wise Accuracy
plt.figure(figsize=(10, 5))
plt.barh(class_names, class_accuracy, color='skyblue')
plt.xlabel("Accuracy")
plt.ylabel("Class")
plt.title("Class-wise Accuracy")
plt.xlim(0.5, 1.0)  # Set x-axis between 50% and 100% for better comparison
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.show()

import random

#  Step 1: Identify Misclassified Samples
misclassified_idx = np.where(y_pred != y_true)[0]
misclassified_samples = random.sample(list(misclassified_idx), 9)

#  Step 2: Plot Misclassified Images
plt.figure(figsize=(10, 10))
for i, idx in enumerate(misclassified_samples):
    plt.subplot(3, 3, i + 1)
    plt.imshow(x_test[idx].reshape(28, 28), cmap='gray')
    plt.title(f"Pred: {class_names[y_pred[idx]]}\nTrue: {class_names[y_true[idx]]}", fontsize=10)
    plt.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

#  Load Your Own Dataset (Example: Fashion MNIST)
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

#  Normalize dataset (scaling pixel values between 0 and 1)
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

#  Define the Baseline Model
def create_baseline_model():
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Define the Enhanced Model
def create_enhanced_model():
    model = keras.Sequential([
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(28, 28, 1)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Train & Evaluate Baseline Model
baseline_model = create_baseline_model()
baseline_history = baseline_model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test), verbose=2)

# Train & Evaluate Enhanced Model
enhanced_model = create_enhanced_model()
enhanced_history = enhanced_model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test), verbose=2)

#  Compare Performance Metrics
baseline_loss, baseline_acc = baseline_model.evaluate(x_test, y_test, verbose=0)
enhanced_loss, enhanced_acc = enhanced_model.evaluate(x_test, y_test, verbose=0)

print("\n *Performance Comparison:*")
print(f" Baseline Model - Accuracy: {baseline_acc*100:.2f}%, Loss: {baseline_loss:.4f}")
print(f" Enhanced Model - Accuracy: {enhanced_acc*100:.2f}%, Loss: {enhanced_loss:.4f}")

# Plot Training & Validation Accuracy & Loss
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

# Accuracy Plot
ax[0].plot(baseline_history.history['accuracy'], 'bo-', label='Baseline Training Acc')
ax[0].plot(baseline_history.history['val_accuracy'], 'b--', label='Baseline Validation Acc')
ax[0].plot(enhanced_history.history['accuracy'], 'ro-', label='Enhanced Training Acc')
ax[0].plot(enhanced_history.history['val_accuracy'], 'r--', label='Enhanced Validation Acc')
ax[0].set_title("Accuracy Comparison")
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Accuracy")
ax[0].legend()
ax[0].grid(True)

# Loss Plot
ax[1].plot(baseline_history.history['loss'], 'bo-', label='Baseline Training Loss')
ax[1].plot(baseline_history.history['val_loss'], 'b--', label='Baseline Validation Loss')
ax[1].plot(enhanced_history.history['loss'], 'ro-', label='Enhanced Training Loss')
ax[1].plot(enhanced_history.history['val_loss'], 'r--', label='Enhanced Validation Loss')
ax[1].set_title("Loss Comparison")
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("Loss")
ax[1].legend()
ax[1].grid(True)

plt.tight_layout()
plt.show()

# Confusion Matrix & Classification Report
y_pred_baseline = np.argmax(baseline_model.predict(x_test), axis=1)
y_pred_enhanced = np.argmax(enhanced_model.predict(x_test), axis=1)

print("\n *Baseline Model Classification Report:*")
print(classification_report(y_test, y_pred_baseline))

print("\n *Enhanced Model Classification Report:*")
print(classification_report(y_test, y_pred_enhanced))

# Plot Confusion Matrices
def plot_confusion_matrix(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=range(10), yticklabels=range(10))
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    plt.title(title)
    plt.show()

plot_confusion_matrix(y_test, y_pred_baseline, "Baseline Model Confusion Matrix")
plot_confusion_matrix(y_test, y_pred_enhanced, "Enhanced Model Confusion Matrix")
